In [178]:
import pandas as pd
import requests
from ipywidgets import IntProgress
from IPython.display import display
import time

In [83]:
houses = pd.read_csv('../raw_data/scrapped_houses.csv')
characters = pd.read_csv('../raw_data/scrapped_characters.csv')
character_predictions = pd.read_csv('../raw_data/character-predictions.csv')
dead = pd.read_excel('../raw_data/only_dead.xlsx', header=1)

In [84]:
houses.columns # Name, Region

Index(['Name', 'Region', 'Coat of Arms', 'Words', 'Titles', 'Seats',
       'Current Lord', 'Heir', 'Overlord', 'Founded', 'Founder', 'Died Out',
       'Ancestral Weapons', 'Cadet Branches', 'Sworn Members', 'URL'],
      dtype='object')

In [85]:
houses = houses[['Name', 'Region']]

In [86]:
characters.columns # Culture, Allegiances

Index(['Name', 'Culture', 'Born', 'Died', 'Titles', 'Aliases', 'Father',
       'Mother', 'Spouse', 'Allegiances', 'Books', 'POV Books', 'TV Series',
       'Played By', 'URL'],
      dtype='object')

In [87]:
characters = characters[['Culture', 'Allegiances']]

In [88]:
character_predictions.columns # culture, house

Index(['S.No', 'actual', 'pred', 'alive', 'plod', 'name', 'title', 'male',
       'culture', 'dateOfBirth', 'DateoFdeath', 'mother', 'father', 'heir',
       'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'boolDeadRelations',
       'isPopular', 'popularity', 'isAlive'],
      dtype='object')

In [89]:
character_predictions = character_predictions[['culture', 'house']]

In [90]:
dead.columns # Allegiance

Index(['Allegiance', 'Death No.', 'Episode', 'Killer', 'Killers House',
       'Location', 'Method', 'Name', 'Season'],
      dtype='object')

In [91]:
dead = dead[['Allegiance']]

In [92]:
houses[['Region']].value_counts()

Region         
The Reach          72
The North          61
The Westerlands    55
The Riverlands     52
The Crownlands     49
Iron Islands       38
The Stormlands     38
The Vale           35
Dorne              29
The Neck            7
Beyond the Wall     1
None                1
dtype: int64

In [93]:
#houses[houses['Region'] == 'The Vale']['Name'].value_counts()

In [95]:
characters['Culture'].value_counts()

Northmen                   137
Ironborn                   127
Free Folk                   70
Valyrian                    57
Braavosi                    46
Dornish                     42
Ghiscari                    35
Dothraki                    23
Rivermen                    22
Valemen                     20
Reach                       16
Westeros                    14
Westerman                   14
Stormlands                  10
Mountain clans               9
Tyroshi                      9
Lysene                       8
First Men                    8
Northern mountain clans      8
Andal                        8
Qartheen                     7
Summer Isles                 7
Myrish                       5
Free folk                    5
Astapori                     5
Pentoshi                     4
Meereenese                   4
Crannogmen                   4
Asshai                       3
Sistermen                    3
Norvoshi                     3
Lhazareen                    3
Rhoynar 

In [128]:
mylist = []
for i in characters['Allegiances']:
    j = i.strip('[]')
    if j:
        k = j.replace("https://www.anapioficeandfire.com/api/houses/", '').\
              replace(' ', '').replace("'", '')
        if ',' in k:
            for l in k.split(','):
                mylist.append(int(l))
        else:
            mylist.append(int(k))

In [217]:
lords = pd.DataFrame(set(sorted(mylist)), columns=['house'])
lords['overlord'] = 0
lords['ol_name'] = ''

In [218]:
lords

,house,overlord,ol_name
0,2,0,
1,4,0,
2,7,0,
3,8,0,
4,10,0,
...,...,...,...
292,439,0,
293,440,0,
294,441,0,
295,443,0,


In [219]:
url = "https://www.anapioficeandfire.com/api/houses/"

In [220]:
f = IntProgress(min=0, max=len(lords)) # instantiate the bar
display(f) # display the bar

for row in range(len(lords)):
    f.value += 1
    i = lords.loc[row, 'house']
    h = requests.get(f"{url}{i}").json()
    ol = h['overlord'].strip("https://www.anapioficeandfire.com/api/houses/")
    if i == 16:
        lords.loc[row, 'overlord'] = 16
        lords.loc[row, 'ol_name'] = h['name']
    elif ol == '16':
        lords.loc[row, 'overlord'] = lords.loc[row, 'house']
        lords.loc[row, 'ol_name'] = h['name']
    else:
        lords.loc[row, 'overlord'] = ol
        if ol != '':
            over = requests.get(f"{url}{int(ol)}").json()
            lords.loc[row, 'ol_name'] = over['name']
        #print(row)

IntProgress(value=0, max=297)

In [221]:
lords[(lords['ol_name'] == 'House Yronwood of Yronwood')]

,house,overlord,ol_name
73,104,444,House Yronwood of Yronwood


In [190]:
test = requests.get(f"{url}{444}").json()

In [194]:
ol = test['overlord'].strip("https://www.anapioficeandfire.com/api/houses/")
ol

'285'

In [195]:
over = requests.get(f"{url}{int(ol)}").json()

In [196]:
over['name']

'House Nymeros Martell of Sunspear'

In [222]:
f = IntProgress(min=0, max=len(lords)) # instantiate the bar
display(f) # display the bar

for row in range(len(lords)):
    f.value += 1
    i = lords.loc[row, 'overlord']
    if i == '':
        continue
    h = requests.get(f"{url}{i}").json()
    ol = h['overlord'].strip("https://www.anapioficeandfire.com/api/houses/")
    if i == 16:
        lords.loc[row, 'overlord'] = 16
        lords.loc[row, 'ol_name'] = h['name']
    elif ol == '16':
        lords.loc[row, 'overlord'] = lords.loc[row, 'overlord']
        lords.loc[row, 'ol_name'] = h['name']
    else:
        lords.loc[row, 'overlord'] = ol
        if ol != '':
            over = requests.get(f"{url}{int(ol)}").json()
            lords.loc[row, 'ol_name'] = over['name']
        #print(row)

IntProgress(value=0, max=297)

In [223]:
row

296

In [225]:
lords['overlord'].value_counts()

398    44
229    37
169    27
10     27
17     25
7      22
362    21
285    17
34     16
       12
15     11
253     1
169     1
175     1
189     1
189     1
378     1
220     1
229     1
243     1
17      1
148     1
285     1
325     1
334     1
354     1
362     1
364     1
386     1
260     1
136     1
117     1
85      1
16      1
15      1
29      1
31      1
10      1
34      1
48      1
49      1
51      1
56      1
57      1
65      1
7       1
69      1
71      1
398     1
Name: overlord, dtype: int64

In [96]:
character_predictions['culture'].value_counts()

Northmen     124
Ironborn     112
Free Folk     51
Valyrian      43
Braavosi      42
            ... 
Andal          1
Norvoshi       1
Qarth          1
Lhazarene      1
The Reach      1
Name: culture, Length: 64, dtype: int64

In [97]:
dead.value_counts()

Allegiance                       
House Lannister                      835
None                                 432
House Stark                          121
Free Folk                            121
Night's Watch                         79
House Targaryen                       68
House Bolton                          68
Sons of the Harpy                     68
House Baratheon of Dragonstone        67
House Frey                            62
Faith Militant                        52
Smallfolk                             44
House Greyjoy                         41
Dothraki                              23
House Baratheon of King's Landing     22
The Masters                           14
The Thirteen                          12
Wise Masters                          12
Brotherhood Without Banners           10
House Martell                          8
House Mormont                          8
Good Masters                           7
House Baelish                          6
Great Masters          